# **TicTacToe Reinforcement Learning and Monte Carlo Methods**



## TicTacToe Environment:

Will be using a custom TicTacToe environment based on the OpenAi gym framework.

**Reward** is a tuple representing the reward each agent receives after the action is performed. The first value is the reward X receives and the second value is the reward O receives. We will use this reward to learn what states are good and bad.

*   If X wins the reward is (10, -10)
*   If O wins the reward is (-10, 10)
*   If the game is a tie or not done yet the reward is (0, 0)


## Setup:


In [1]:
pip install gym==0.20.0


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
!unzip -o gym-tictactoe.zip

Archive:  gym-tictactoe.zip
  inflating: gym-tictactoe/setup.py  
  inflating: gym-tictactoe/gym_tictactoe.egg-info/PKG-INFO  
  inflating: gym-tictactoe/gym_tictactoe.egg-info/SOURCES.txt  
  inflating: gym-tictactoe/gym_tictactoe.egg-info/requires.txt  
  inflating: gym-tictactoe/gym_tictactoe.egg-info/top_level.txt  
  inflating: gym-tictactoe/gym_tictactoe.egg-info/dependency_links.txt  
  inflating: gym-tictactoe/gym_tictactoe/__init__.py  
  inflating: gym-tictactoe/.ipynb_checkpoints/setup-checkpoint.py  
  inflating: gym-tictactoe/gym_tictactoe/__pycache__/__init__.cpython-39.pyc  
  inflating: gym-tictactoe/gym_tictactoe/.ipynb_checkpoints/__init__-checkpoint.py  
  inflating: gym-tictactoe/gym_tictactoe/envs/__init__.py  
  inflating: gym-tictactoe/gym_tictactoe/envs/tictactoe_env.py  
  inflating: gym-tictactoe/gym_tictactoe/envs/__pycache__/__init__.cpython-39.pyc  
  inflating: gym-tictactoe/gym_tictactoe/envs/__pycache__/tictactoe_env.cpython-39.pyc  
  inflating: gym-tic

In [4]:
pip install -e gym-tictactoe

Obtaining file:///Users/artem/Downloads/gym-tictactoe
  Preparing metadata (setup.py) ... done
  Running setup.py develop for gym-tictactoe

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Importing Required Libraries 

In [1]:
# Gym is a library that will allow us to initialize and work with the TicTacToe environment
import gym

# Random allows us to make random choices when interacting with the environment
import random
import numpy as np

# Custom tictactoe environment
import gym_tictactoe

## Experiment

Initialize the environment:

In [3]:
env = gym.make('TicTacToe-v0')

Visualize the environment:

In [6]:
env.render()

Board
['-', '-', '-']
['-', '-', '-']
['-', '-', '-']


Check the **action space** of the environment.

In [7]:
env.available_actions()

[0, 1, 2, 3, 4, 5, 6, 7, 8]

Make a step:

In [10]:
new_state, reward, done, info = env.step(0, "X")
new_state, reward

('X--------', (0, 0))

Define a new python function that will follow the epsilon probability and return an action:

In [12]:
def random_action(action,epsilon=0.1,avb_actions=[]):
    ''' 
    This function takes the best estimated action, eplsilon, and action space 
    and returns some action. 
    '''
    # generate a random number from 0 to 1.
    number = np.random.rand(1)
    
    # if number is smaller than 1-epsilon then return best estimated action
    if number<1-epsilon:
        return action
    # if number is bigger or equals to 1-epsilon then return some random action from the action space
    else:
        action=random.choice(avb_actions)  
        return action 

## Monte Carlo Algorithm - Train X

Implement the MC algorithm, a few things to note: 
* We will be training $Agent_X$, so the one who plays as "X" on the board.
* We will be training our agent against a random oponent (so oponent, $Agent_O$, will make random actions).

In [13]:
def monte_carlo_X(environment,N_episodes=10000,epsilon=0.1, discount_factor=1):
    """
    This function determines the optimal policy usistate=environment.reset() ng the epsilon greedy method. The input parameters are the discount factor, the number of episodes, epsilon value and the open AI gym objects. The class also Specifies if first-visit and every-visit method. The output is the policy,value function and action function.
    Returns:  
    policy 
    Q funciton
    
    Args:
    environment:AI gym balckjack envorment object 
    epsilon: the exploration factor
    N_episodes:number of episodes 
    discount_factor:discount factor
    """
    #dictionary of estimated action function for FrozenLake
    Q={}
    
    # number of visits to the action function 
    NumberVisits= {}
   
    #dictionary  for policy 
    policy={}
    
    number_actions = 9
    
    for i in range(N_episodes):
        
        #list that stores each state and reward for each episode     
        episode=[]
        # reset the  environment for the next episode and find first state  
        environment.reset() 
        state = environment.hash()
        
        #flag for end of episodes  
        done=False
        
        #check if a policy for the state exists  
        if state in policy.keys():
            #obtain action from policy 
            action= policy[state]
            action= random_action(action,epsilon,environment.available_actions())

        else:
            #if no policy for the state exists  select a random  action  
            action=random.choice(environment.available_actions())
            
        #take action and find next state, reward
        (state_1, reward, done, info) = environment.step(action, "X")
        #append first state, reward and action
        episode.append({'state':state, 'reward':reward[0],'action':action})
        
        #enumerate for each episode 
        while not(done):
            #Agent O should make a random action.
            (state, reward, done, info) = environment.step(random.choice(environment.available_actions()), "O")
            
            #Check if the episode is done (True) 
            if not done:
                #Check if a policy for the state exists 
                if state in policy.keys():
                    #obtain action from policy 
                    action=policy[state]
                    action= random_action(action,epsilon,environment.available_actions())

                else:
                    #if no policy for the state exists  select a random  action  
                    action=random.choice(environment.available_actions())
                #take action and find next state, reward
                (state_1, reward, done, info) = environment.step(action, "X")
                
            #add state reward and action to list 
            episode.append({'state':state, 'reward':reward[0],'action':action})
        
        #reverse list as the return is calculated from the last state
        episode.reverse()

        # determine the return
        G=0
        for t,step in enumerate(episode):
                
                G=discount_factor*G+step['reward']
                
                #increment counter for action 
                if (step['state'],step['action']) in NumberVisits.keys():
                    #obtain action from policy 
                    NumberVisits[step['state'],step['action']]+=1

                else:
                    #if no policy for the state exists  select a random  action  
                    NumberVisits[step['state'],step['action']]=1
        
                #if the action function value  does not exist, create an array  to store them 
                if not step['state'] in Q.keys():
                    Q[step['state']]= np.zeros((number_actions))

                #calculate mean of action function Q Value functions using the  recursive definition of mean 
                Q[step['state']][step['action']]=Q[step['state']][step['action']]+(NumberVisits[step['state'],step['action']]**-1)*(G-Q[step['state']][step['action']])
                
                #setting the value of impossible statee-action pairs to -1000 (not the most elegant but the most efficient way in our simple case)
                for i,x in enumerate(step['state']):
                    if x != "-":
                        Q[step['state']][i] = -1000
                        
                #update the policy to select the action fuciton argment with the largest value randomly select a different action 
                policy[step['state']]=np.random.choice(np.where(Q[step['state']]==Q[step['state']].max())[0])
                        
   
    return policy, Q

Train it and see the results: 

In [15]:
policy_X, Q= monte_carlo_X(env,N_episodes=100000,epsilon=0.1, discount_factor=0.95)  

Check the results of training and analyze how agent performs, for the following code the actions of X player will be dictated only by our policy, let's see the results:

In [16]:
total = 0
test = 1000
for i in range(test):
    # variable to keep track of if the game is over
    done = False
    # Good practice to reset environment before you play a game to clear any old game
    env.reset()
    # Want to keep playing untill game is over
    new_state = env.hash()
    while not done:
        # Make an action from policy for X
        new_state, reward, done, info = env.step(policy_X[new_state], "X")

        # If the game is done on X action we dont want O to make an action
        if not done:
            # Make a random action from the list of available actions for O
            new_state, reward, done, info = env.step(random.choice(env.available_actions()), "O")
        
    total += reward[0]
        
# Print the reward after the game is done, reward for X is the first value and O is the second value
print(f"{total/test *10}%")

97.8%


Agent performs extraordinary well against random agent.

## Monte Carlo Algorithm - Train O

Training notes: 
* We will be training $Agent-O$, so the one who plays as "O" on the board.
* We will be training our agent against a random oponent (so oponent, $Agent-X$, will make random actions).

In [19]:
def monte_carlo_O(environment,N_episodes=10000,epsilon=0.1, discount_factor=1):
    """
    This function determines the optimal policy usistate=environment.reset() ng the epsilon greedy method. The input parameters are the discount factor, the number of episodes, epsilon value and the open AI gym objects. The class also Specifies if first-visit and every-visit method. The output is the policy,value function and action function.
    Returns:  
    policy 
    Q funciton
    
    Args:
    environment:AI gym balckjack envorment object 
    epsilon: the exploration factor
    N_episodes:number of episodes 
    discount_factor:discount factor
    """
    #dictionary of estimated action function for FrozenLake
    Q={}
    
    # number of visits to the action function 
    NumberVisits= {}
   
    #dictionary  for policy 
    policy={}
    
    #number of possible actions
    number_actions = 9
    
    for i in range(N_episodes):
        
        #list that stores each state and reward for each episode     
        episode=[]
        #reset the  environment for the next episode and find first state  
        environment.reset() 
        state = environment.hash()
        
        #flag for end of episodes  
        done=False
        
        #enumerate for each episode 
        while not(done):
            #Agent X should make a random action
            (state_1, reward, done, info) = environment.step(random.choice(environment.available_actions()), "X")
            
            #Check if the episode is done (True) 
            if not done: 
                if state_1 in policy.keys():
                    #obtain action from policy 
                    action=policy[state_1]
                    action= random_action(action,epsilon,environment.available_actions())

                else:
                    #if no policy for the state exists  select a random  action  
                    action=random.choice(environment.available_actions())
                #Agent O should make a given aciton.
                (state, reward, done, info) = environment.step(action, "O")
                
                    
            #add state reward and action to list 
            episode.append({'state':state_1, 'reward':reward[1],'action':action})
        
        #reverse list as the return is calculated from the last state
        episode.reverse()

        # determine the return
        G=0
        for t,step in enumerate(episode):
                
                G=discount_factor*G+step['reward']
                
                #increment counter for action 
                if (step['state'],step['action']) in NumberVisits.keys():
                    #obtain action from policy 
                    NumberVisits[step['state'],step['action']]+=1

                else:
                    #if no policy for the state exists  select a random  action  
                    NumberVisits[step['state'],step['action']]=1
        
                #if the action function value  does not exist, create an array  to store them 
                if not step['state'] in Q.keys():
                    Q[step['state']]= np.zeros((number_actions))

                #calculate mean of action function Q Value functions using the  recursive definition of mean 
                Q[step['state']][step['action']]=Q[step['state']][step['action']]+(NumberVisits[step['state'],step['action']]**-1)*(G-Q[step['state']][step['action']])
                 
                #setting the value of impossible state-action pairs to -1000 (not the most elegant but the most efficient way in our simple case)
                for i,x in enumerate(step['state']):
                    if x != "-":
                        Q[step['state']][i] = -1000
                
                #update the policy to select the action fuciton argment with the largest value randomly select a different action 
                policy[step['state']]=np.random.choice(np.where(Q[step['state']]==Q[step['state']].max())[0])
                        
   
    return policy, Q

Train it as well and see the results:

In [20]:
policy_O, Q = monte_carlo_O(env,N_episodes=100000,epsilon=0.1, discount_factor=1)

 Run it against the random X player: 

In [22]:
total = 0
test = 1000
for i in range(test):
    # variable to keep track of if the game is over
    done = False
    # Good practice to reset environment before you play a game to clear any old game
    env.reset()
    # Want to keep playing untill game is over
    new_state = env.hash()
    while not done:
        # Make a random action from the list of available actions for X
        new_state, reward, done, info = env.step(random.choice(env.available_actions()), "X")

        # If the game is done on X action we dont want O to make an action
        if not done:
            # Make an action from the list of available actions for O
            new_state, reward, done, info = env.step(policy_O[new_state], "O")
        
    total += reward[1]
        
    # Print the reward after the game is done, reward for X is the first value and O is the second value
print(f"{total/test *10}%")

86.4%


Try putting one trained agent against another: 

In [25]:

def test(episodes):
    # counters to keep track of results
    x_win = 0
    o_win = 0
    tie = 0
    # loops for a certain number of games (episodes)
    for episode in range(episodes):
        # stops while loop when game is done
        done = False
        # resets environment when game is done
        env.reset()
        new_state = env.hash()
        # X agents turn
        # performs an action
        new_state, reward, done, _ = env.step(policy_X[new_state], "X")
        while not done:
            # O agents turn
            # Get the best action
            new_state, reward, done, _ = env.step(policy_O[new_state], "O")
            
            # if the game ends on O move, we don't want to make an X move
            if (not done):
                 # X agents turn
                 # performs an action
                new_state, reward, done, _ = env.step(policy_X[new_state], "X")
        # record results when game is done
        if (reward == (-10, 10)):
            o_win+=1
        elif (reward == (10, -10)):
            x_win+=1
        elif (reward == (0, 0)):
            tie+=1
    return x_win, o_win, tie

episodes = 10000

x_win, o_win, tie = test(episodes)

print("X Win:", x_win, "Tie:", tie, "O Win:", o_win)
print("X Win Rate:", x_win/episodes*100, "Tie Rate:", tie/episodes*100, "O Win Rate:", o_win/episodes*100)

X Win: 0 Tie: 10000 O Win: 0
X Win Rate: 0.0 Tie Rate: 100.0 O Win Rate: 0.0


The function returns a perfect Tie Rate of $100\%$ as expected.

**Conclusion** : have created a successful algorithm, that solves a deterministic environment, against a similary trained agent.

### Resources

* [ Algorithms and pseudocode: Sutton, Richard S., and Andrew G. Barto. "Reinforcement learning: An introduction"](https://www.amazon.ca/Reinforcement-Learning-Introduction-Second-Paperback/dp/B0B95WFGV6/ref=asc_df_B0B95WFGV6/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsIBMDeveloperSkillsNetworkGPXX0HAUEN1421-2022-01-01&tag=googleshopc0c-20&linkCode=df0&hvadid=578919340205&hvpos=&hvnetw=g&hvrand=11011609051689383259&hvpone=&hvptwo=&hvqmt=&hvdev=c&hvdvcmdl=&hvlocint=&hvlocphy=9000828&hvtargid=pla-1728961664549&psc=1)